**Description**: Temporarily open pretrained EC and check why its TPR/TNR isn't what I expected. Probably because threshold wasn't taken properly.

In [1]:
from importlib import reload
import gnn_tracking_hpo
reload(gnn_tracking_hpo)

from gnn_tracking_hpo.util.paths import add_scripts_path

add_scripts_path()

In [10]:
from tune_fixed_ec_sectorized import load_ec, PretrainedECTrainable

In [5]:
ec = load_ec("ec-s9", "009d", config_update={"n_graphs_train": 1, "n_graphs_val": 100, "n_graphs_test": 1})

DEBUG: Got config
{'batch_size': 1,
 'focal_alpha': 0.7397820322968228,
 'focal_gamma': 4,
 'gnn_tracking_experiments_hash': '1c4385064cb8472d7070c92d38d2958ab96e7485',
 'gnn_tracking_hash': 'd1903e7319ef1dc27f2632d27212ce802d1273e2',
 'lr': 0.00010008424753725798,
 'lw_edge': 1.0,
 'm_L_ec': 7,
 'm_alpha_ec': 0.5082980468439962,
 'm_e_dim': 4,
 'm_h_dim': 5,
 'm_hidden_dim': 118,
 'm_interaction_edge_hidden_dim': 85,
 'm_interaction_node_hidden_dim': 94,
 'n_graphs_test': 1,
 'n_graphs_train': 1,
 'n_graphs_val': 1,
 'optimizer': 'adam',
 'scheduler': None,
 'sector': 9,
 'test': False,
 'training_pt_thld': 0.0,
 'training_without_noise': False,
 'training_without_non_reconstructable': False}
INFO: Loading data to cpu memory
INFO: Loading 3 graphs (out of 371 available).
DEBUG: Parameters for data loaders: {'batch_size': 1, 'num_workers': 12}
INFO: Using device cpu


In [14]:
from gnn_tracking_hpo.trainable import suggest_default_values
from typing import Any
import optuna
from gnn_tracking_hpo.config import auto_suggest_if_not_fixed, get_metadata


def suggest_config(
    trial: optuna.Trial,
    *,
    sector: int,
    test=False,
    fixed= None,
) -> dict[str, Any]:
    config = get_metadata(test=test)
    config.update(fixed or {})

    def d(key, *args, **kwargs):
        auto_suggest_if_not_fixed(key, config, trial, *args, **kwargs)

    d("sector", sector)
    d("n_graphs_train", 300)
    d("n_graphs_val", 69)
    d("n_graphs_test", 1)

    d("ec_project", "ec-s9")
    d("ec_hash", "009d")
    d("m_ec_threshold", 0.3746)

    suggest_default_values(config, trial, ec="fixed")
    return config

In [15]:
config =suggest_config(None, sector=9)

In [16]:
config

{'test': False,
 'gnn_tracking_hash': 'c3886f63ced02fd529f64ab7a0b357b8f5a59b06',
 'gnn_tracking_experiments_hash': '9f2524d3edff18c30632f4de1334782ecfadbd38',
 'sector': 9,
 'n_graphs_train': 300,
 'n_graphs_val': 69,
 'n_graphs_test': 1,
 'ec_project': 'ec-s9',
 'ec_hash': '009d',
 'm_ec_threshold': 0.3746,
 'training_pt_thld': 0.0,
 'training_without_noise': False,
 'training_without_non_reconstructable': False,
 'batch_size': 1,
 'm_interaction_node_hidden_dim': 5,
 'm_interaction_edge_hidden_dim': 4,
 'repulsive_radius_threshold': 10.0,
 'q_min': 0.01,
 'attr_pt_thld': 0.9,
 'sb': 0.1,
 'lr': 0.0005,
 'optimizer': 'adam',
 'scheduler': None,
 'm_h_dim': 5,
 'm_e_dim': 4,
 'm_h_outdim': 2,
 'm_hidden_dim': 40,
 'm_L_hc': 3,
 'm_alpha_hc': 0.5}

In [17]:
trainable = PretrainedECTrainable(config)

DEBUG: Got config
{'batch_size': 1,
 'focal_alpha': 0.7397820322968228,
 'focal_gamma': 4,
 'gnn_tracking_experiments_hash': '1c4385064cb8472d7070c92d38d2958ab96e7485',
 'gnn_tracking_hash': 'd1903e7319ef1dc27f2632d27212ce802d1273e2',
 'lr': 0.00010008424753725798,
 'lw_edge': 1.0,
 'm_L_ec': 7,
 'm_alpha_ec': 0.5082980468439962,
 'm_e_dim': 4,
 'm_h_dim': 5,
 'm_hidden_dim': 118,
 'm_interaction_edge_hidden_dim': 85,
 'm_interaction_node_hidden_dim': 94,
 'n_graphs_test': 1,
 'n_graphs_train': 1,
 'n_graphs_val': 1,
 'optimizer': 'adam',
 'scheduler': None,
 'sector': 9,
 'test': False,
 'training_pt_thld': 0.0,
 'training_without_noise': False,
 'training_without_non_reconstructable': False}
INFO: Loading data to cpu memory
INFO: Loading 3 graphs (out of 371 available).
DEBUG: Parameters for data loaders: {'batch_size': 1, 'num_workers': 12}
INFO: Using device cpu
DEBUG: Got config
{'attr_pt_thld': 0.9,
 'batch_size': 1,
 'ec_hash': '009d',
 'ec_project': 'ec-s9',
 'gnn_tracking_expe

In [19]:
from gnn_tracking.training.tcn_trainer import TCNTrainer

TCNTrainer.test_step(trainable.trainer)

INFO: Starting hyperparameter scan for clustering
INFO: Completed 12 trials, pruned 0 trials
DEBUG: Evaluating all metrics for best clustering
INFO: Evaluating all metrics took 2.61 seconds
INFO: Clustering hyperparameter scan & metric evaluation took 29.15 seconds


{'acc_pt0.9': 0.9912747145993022,
 'TPR_pt0.9': 0.6723429497390527,
 'TNR_pt0.9': 0.997777171669292,
 'FPR_pt0.9': 0.0022228283307081514,
 'FNR_pt0.9': 0.3276570502609471,
 'roc_auc_pt0.9': 0.9792254648404642,
 'roc_auc_5FPR_pt0.9': 0.889410940659797,
 'roc_auc_10FPR_pt0.9': 0.9173466632236574,
 'roc_auc_20FPR_pt0.9': 0.9463588983528485,
 'acc_pt1.5': 0.991479963683378,
 'TPR_pt1.5': 0.6536869187724439,
 'TNR_pt1.5': 0.9976241054943649,
 'FPR_pt1.5': 0.0023758945056351058,
 'FNR_pt1.5': 0.34631308122755605,
 'roc_auc_pt1.5': 0.9778350160981233,
 'roc_auc_5FPR_pt1.5': 0.8837895622360141,
 'roc_auc_10FPR_pt1.5': 0.9122333444573301,
 'roc_auc_20FPR_pt1.5': 0.9425449656334431,
 'total': 726.3389242421026,
 'potential_attractive': 1.0681779634108787e-05,
 'potential_attractive_weighted': 1.0681779634108787e-05,
 'potential_repulsive': 725.8389910269475,
 'potential_repulsive_weighted': 725.8389910269475,
 'background': 0.49993171078571375,
 'background_weighted': 0.49993171078571375,
 'v_me

In [28]:
trainable.trainer.model._gtcn.threshold

ModularGraphTCN(
  (relu): ReLU()
  (ec): UnPackDictionaryForward(
    (ec): SignatureAdaptedECForGraphTCN(
      (relu): ReLU()
      (ec_node_encoder): MLP(
        (layers): ModuleList(
          (0): Linear(in_features=6, out_features=118, bias=False)
          (1): ReLU()
          (2): Linear(in_features=118, out_features=5, bias=False)
        )
      )
      (ec_edge_encoder): MLP(
        (layers): ModuleList(
          (0): Linear(in_features=4, out_features=118, bias=False)
          (1): ReLU()
          (2): Linear(in_features=118, out_features=4, bias=False)
        )
      )
      (ec_resin): ResIN(
        (layers): ModuleList(
          (0): InteractionNetwork()
          (1): InteractionNetwork()
          (2): InteractionNetwork()
          (3): InteractionNetwork()
          (4): InteractionNetwork()
          (5): InteractionNetwork()
          (6): InteractionNetwork()
        )
        (residue_encoders): ModuleList(
          (0): MLP(
            (layers): Modu